# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

#Paso 1. Hacer la solicitud a la URL
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_data = requests.get(url, time.sleep(15)).text

# If no information is extracted, then connect as anonymous
if "403 Forbidden" in html_data:
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"}
    request = requests.get(url, headers = headers)
    time.sleep(15)
    html_data = request.text

html_data
print("PASO 1.", html_data)



PASO 1. <html>
<head>

</head>
<body>
<div style="margin: 50px auto; width: 50%; border: 1px solid #dfdfdf; padding: 20px 50px 30px 50px; font-family:helvetica;">

<h1>We do not allow automated access to our servers.</h1>
<h2><p>Automated access to our data is prohibited by our data provider.</p>  
<p>If you are a user attempting to access the site via a browser, please follow this process to regain access:</p>

<ul>
  <li>Go to <a href='https://whatismyipaddress.com/' target='_blank' rel='noopener noreferrer'>whatismyipaddress</a> and obtain your IPv4 address</li>
  <li>Email us your IPv4 address at <a href="/cdn-cgi/l/email-protection" class="__cf_email__" data-cfemail="4821262e270825292b3a273c3a2d262c3b66262d3c">[email&#160;protected]</a></li>
  <li>We will add you to our whitelist within 24 hours</li>
</ul>
</h2>
</div>
<script data-cfasync="false">!function(){"use strict";function e(e){try{if("undefined"==typeof console)return;"error"in console?console.error(e):console.log(e)}catc

In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

#Paso 1. Hacer la solicitud a la URL
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_data = requests.get(url, time.sleep(15)).text

# If no information is extracted, then connect as anonymous
if "403 Forbidden" in html_data:
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"}
    request = requests.get(url, headers = headers)
    time.sleep(15)
    html_data = request.text

#Paso 2. Parsear el HTML con BeutifulSoup
soup = BeautifulSoup(html_data, 'html.parser')
soup
print("PASO 2.", soup)


PASO 2. <html>
<head>
</head>
<body>
<div style="margin: 50px auto; width: 50%; border: 1px solid #dfdfdf; padding: 20px 50px 30px 50px; font-family:helvetica;">
<h1>We do not allow automated access to our servers.</h1>
<h2><p>Automated access to our data is prohibited by our data provider.</p>
<p>If you are a user attempting to access the site via a browser, please follow this process to regain access:</p>
<ul>
<li>Go to <a href="https://whatismyipaddress.com/" rel="noopener noreferrer" target="_blank">whatismyipaddress</a> and obtain your IPv4 address</li>
<li>Email us your IPv4 address at <a class="__cf_email__" data-cfemail="4821262e270825292b3a273c3a2d262c3b66262d3c" href="/cdn-cgi/l/email-protection">[email protected]</a></li>
<li>We will add you to our whitelist within 24 hours</li>
</ul>
</h2>
</div>
<script data-cfasync="false">!function(){"use strict";function e(e){try{if("undefined"==typeof console)return;"error"in console?console.error(e):console.log(e)}catch(e){}}function 

In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

# URL de la página web
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

# Paso 1. Hacer la solicitud a la URL
response = requests.get(url)
time.sleep(15)  # Esperar 15 segundos

html_data = response.text

# Si no se extrae información, conectarse como anónimo
if "403 Forbidden" in html_data:
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, como Gecko) Chrome/50.0.2661.102 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    time.sleep(15)  # Esperar 15 segundos
    html_data = response.text

# Parsear el HTML
soup = BeautifulSoup(html_data, 'html.parser')

# Encontrar todas las tablas dentro de los <div> con clase 'row'
rows = soup.find_all('div', class_='row')

# Lista para almacenar los datos extraídos
all_data = []

# Iterar sobre cada <div> con clase 'row'
for row_div in rows:
    col_divs = row_div.find_all('div', class_='col-6')
    for table_div in col_divs:
        table = table_div.find('table', class_='table')  # Encontrar la tabla dentro del div
        if table:
            # Extraer las filas de la tabla
            rows = table.find('tr')
            header = [th.text.strip() for th in rows[0].find_all('th')]  # Extraer encabezados
            data_rows = rows[1:]  # Ignorar la fila de encabezado
            
            # Iterar sobre las filas de datos
            for row in data_rows:
                cols = row.find_all('td')
                if len(cols) > 1:  # Verificar que la fila tiene dos celdas
                    date = cols[0].text.strip()  # Fecha
                    value = cols[1].text.strip()  # Valor
                    all_data.append([date, value])
        else:
            print("No table found in this div.")

if not all_data:
    print("No data found in any table.")
else:
    # Crear un DataFrame con los datos extraídos
    df = pd.DataFrame(all_data, columns=["Date", "Value"])

    # Mostrar el DataFrame
    print(df)

# Verificar las filas de las tablas
for row_div in rows:
    col_divs = row_div.find_all('div', class_='col-6')
    for table_div in col_divs:
        table = table_div.find('table', class_='table')  
        if table:
            rows = table.find_all('tr')
            print(f"Filas encontradas: {len(rows)}")
            for row in rows:
                print(row.text.strip())  # Imprime las filas para depuración


No data found in any table.
